## Unsupervised Learning Capstone:

For this project you'll dig into a large amount of text and apply most of what you've covered in this unit and in the course so far.

First, pick a set of texts. This can be either a series of novels, chapters, or articles. Anything you'd like. It just has to have multiple entries of varying characteristics. At least 100 should be good. There should also be at least 10 different authors, but try to keep the texts related (either all on the same topic of from the same branch of literature - something to make classification a bit more difficult than obviously different subjects).

This capstone can be an extension of your NLP challenge if you wish to use the same corpus. If you found problems with that data set that limited your analysis, however, it may be worth using what you learned to choose a new corpus. Reserve 25% of your corpus as a test set.

The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

Lastly return to your holdout group. Does your clustering on those members perform as you'd expect? Have your clusters remained stable or changed dramatically? What about your model? Is it's performance consistent?

If there is a divergence in the relative stability of your model and your clusters, delve into why.

Your end result should be a write up of how clustering and modeling compare for classifying your texts. What are the advantages of each? Why would you want to use one over the other? Approximately 3-5 pages is a good length for your write up, and remember to include visuals to help tell your story!

In [1]:
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import time

import nltk
import spacy
import re
%matplotlib inline

In [2]:
nltk.download('reuters')
from nltk.corpus import reuters, stopwords
#print(reuters.fileids())

[nltk_data] Downloading package reuters to
[nltk_data]     /Users/brianmcguckin/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [9]:
#each file is pre-labeled for train (~73%) and test (remaining 27%)
train_files = list(filter(lambda x: x.startswith('training'), reuters.fileids()))
test_files = list(filter(lambda x: x.startswith('test'), reuters.fileids()))

print(len(train_files))
print(len(test_files))

7769
3019


In [10]:
train_cats = reuters.categories(train_files)
test_cats = reuters.categories(test_files)

#does the test set contain any categories that the train set doesn't?
print([x for x in test_cats if x not in train_cats])

[]


In [11]:
#see what this data looks like
train_raw = reuters.raw(train_files)
test_raw = reuters.raw(test_files)

print(train_raw[:100])

BAHIA COCOA REVIEW
  Showers continued throughout the week in
  the Bahia cocoa zone, alleviating th


In [13]:
train_files[:5]

['training/1',
 'training/10',
 'training/100',
 'training/1000',
 'training/10000']

In [18]:
asasdf = reuters.raw('training/1')
print(asasdf[:100])

BAHIA COCOA REVIEW
  Showers continued throughout the week in
  the Bahia cocoa zone, alleviating th


In [23]:
print(len(reuters.words(train_files)))
print(len(reuters.raw(train_files)))
print(len(reuters.words(test_files)))
print(len(reuters.raw(test_files)))

1253696
6478471
467205
2368382


In [35]:
train_df = pd.DataFrame(columns=['fileid', 'raw', 'words', 'category'])
for file in train_files:
    train_df = train_df.append({
        'fileid':file,
        'raw':reuters.raw([file]),
        'words':reuters.words(file),
        'category':reuters.categories(file)}, ignore_index=True)
print(len(train_files), train_df.shape)

test_df = pd.DataFrame(columns=['fileid', 'raw', 'words', 'category'])
for file in test_files:
    test_df = test_df.append({
        'fileid':file,
        'raw':reuters.raw(file),
        'words':reuters.words(file),
        'category':reuters.categories(file)}, ignore_index=True)
print(len(test_files), test_df.shape)

7769 (7769, 4)
3019 (3019, 4)


In [34]:
test_df.head(10)

,fileid,raw,words,category
0,test/14826,ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...,"(ASIAN, EXPORTERS, FEAR, DAMAGE, FROM, U, ., S...",[trade]
1,test/14828,CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...,"(CHINA, DAILY, SAYS, VERMIN, EAT, 7, -, 12, PC...",[grain]
2,test/14829,JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...,"(JAPAN, TO, REVISE, LONG, -, TERM, ENERGY, DEM...","[crude, nat-gas]"
3,test/14832,THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n ...,"(THAI, TRADE, DEFICIT, WIDENS, IN, FIRST, QUAR...","[corn, grain, rice, rubber, sugar, tin, trade]"
4,test/14833,INDONESIA SEES CPO PRICE RISING SHARPLY\n Ind...,"(INDONESIA, SEES, CPO, PRICE, RISING, SHARPLY,...","[palm-oil, veg-oil]"
5,test/14839,AUSTRALIAN FOREIGN SHIP BAN ENDS BUT NSW PORTS...,"(AUSTRALIAN, FOREIGN, SHIP, BAN, ENDS, BUT, NS...",[ship]
6,test/14840,INDONESIAN COMMODITY EXCHANGE MAY EXPAND\n Th...,"(INDONESIAN, COMMODITY, EXCHANGE, MAY, EXPAND,...","[coffee, lumber, palm-oil, rubber, veg-oil]"
7,test/14841,SRI LANKA GETS USDA APPROVAL FOR WHEAT PRICE\n...,"(SRI, LANKA, GETS, USDA, APPROVAL, FOR, WHEAT,...","[grain, wheat]"
8,test/14842,WESTERN MINING TO OPEN NEW GOLD MINE IN AUSTRA...,"(WESTERN, MINING, TO, OPEN, NEW, GOLD, MINE, I...",[gold]
9,test/14843,SUMITOMO BANK AIMS AT QUICK RECOVERY FROM MERG...,"(SUMITOMO, BANK, AIMS, AT, QUICK, RECOVERY, FR...",[acq]


In [7]:
def text_cleaner(text):
    text = re.sub(r'--',' ',text)
    text = re.sub('[\[].*?[\]]','',text)
    text = ' '.join(text.split())
    return text

train = text_cleaner(train_raw)
test = text_cleaner(test_raw)

print(len(train))
print(len(test))

6064562
2197712


## tf-idf feature generation

In [ ]:
from sklearn.feature_extraction.text

## Tokenization with Spacy

In [ ]:
#conform with spacys character limit
train_substrings = [train[i:i+999999] for i in range(0, len(train), 999999)]
test_substrings = [test[i:i+999999] for i in range(0, len(test), 999999)]

In [ ]:
train0 = train_substrings[0]
train1 = train_substrings[1]
train2 = train_substrings[2]
train3 = train_substrings[3]
train4 = train_substrings[4]
train5 = train_substrings[5]
train6 = train_substrings[6]

test0 = test_substrings[0]
test1 = test_substrings[1]
test2 = test_substrings[2]

In [ ]:
nlp = spacy.load('en')

start = time.clock()
train0_doc = nlp(train0)
print(time.clock() - start)
train1_doc = nlp(train1)
print(time.clock() - start)
train2_doc = nlp(train2)
print(time.clock() - start)
train3_doc = nlp(train3)
print(time.clock() - start)
train4_doc = nlp(train4)
print(time.clock() - start)
train5_doc = nlp(train5)
print(time.clock() - start)
train6_doc = nlp(train6)
print(time.clock() - start)
test0_doc = nlp(test0)
print(time.clock() - start)
test1_doc = nlp(test1)
print(time.clock() - start)
test2_doc = nlp(test2)
print(time.clock() - start)

In [ ]:
train_docs = [train0_doc, train1_doc, train2_doc, train3_doc, train4_doc, train5_doc, train6_doc]
train_tokens = []
for doc in train_docs:
    for token in doc:
        train_tokens.append(token)
        
test_docs = [test0_doc, test1_doc, test2_doc]
test_tokens = []
for doc in test_docs:
    for token in doc:
        test_tokens.append(token)

In [ ]:
#make sure this worked
print(len(train_tokens))
print(len(train0_doc) + len(train1_doc) + len(train2_doc) + len(train3_doc)
      + len(train4_doc) + len(train5_doc) + len(train6_doc))

print(len(test_tokens))
print(len(test0_doc) + len(test1_doc) + len(test2_doc))